## Introduction to `robotoc` 
# 3: Optimal Control of a Robot Manipulator

## 本章の目的
ここでは，ロボットマニピュレータ KUKA iiwa14 のシンプルな最適制御問題（OCP）を定式化し，解くことが目的です．

KUKA iiwa14 の `robotoc.Robot` モデルを作成します．

In [ ]:
import robotoc
path_to_urdf = "urdf/iiwa_description/urdf/iiwa14.urdf"
iiwa14 = robotoc.Robot(path_to_urdf)

iiwa14の関節角速度およびトルクの上限を設定します．    
(この値は制御などの目的により変わると思います．`robotoc`のAPIから
好きに設定できます．)

In [ ]:
import numpy as np
iiwa14.set_joint_effort_limit(np.full(iiwa14.dimu(), 50))
iiwa14.set_joint_velocity_limit(np.full(iiwa14.dimv(), 0.5*np.pi))

それらの設定した値が反映されているか以下で確認できます．

In [ ]:
print(iiwa14)

## Cost Function（評価関数）
次に，評価関数 (cost function) を作成します．   
`robotoc`では，様々な評価関数のコンポーネント（**cost components**）を組み合わせて評価関数, `robotoc.CostFunction` (`std::shared_ptr<robotoc::CostFunction>` in C++), を作ります．  
各 **cost component** はある特定の評価関数の項を表しており，`robotoc.CostFunctionComponentBase` (`robotoc::CostFunctionComponentBase` in C++) を継承しています.   
**cost component** を評価関数 `robotoc.CostFunction` に集め， 評価関数 `robotoc.CostFunction` をソルバーに渡します．   
このようにすることで，複数の**cost components** を簡単に組み合わせて評価関数が設計できるようになっています.

基本的な **cost components** は `robotoc` のパッケージに既に多数実装されています.  
この例では， configuration つまり関節角，角速度，トルクなどについての**cost component**を考えます．   
以下がその **cost component** です．

In [ ]:
config_cost = robotoc.ConfigurationSpaceCost(iiwa14)

レファレンスと重みパラメータを設定していきます．   
以下のような命名規則があります．   
- `q`: configuration (関節角)
- `v`: 一般化速度 (関節角速度)
- `a`: 一般化加速度 (関節角加速度)
- `u`: 関節トルク
- `qf`, `vf`: ホライゾン終端での関節角度と角速度

In [ ]:
q_ref = np.array([0, 0.5*np.pi, 0, 0.5*np.pi, 0, 0.5*np.pi, 0]) 
config_cost.set_q_ref(q_ref)
config_cost.set_q_weight(np.full(iiwa14.dimv(), 10))
config_cost.set_qf_weight(np.full(iiwa14.dimv(), 10))
config_cost.set_v_weight(np.full(iiwa14.dimv(), 0.01))
config_cost.set_vf_weight(np.full(iiwa14.dimv(), 0.01))
config_cost.set_a_weight(np.full(iiwa14.dimv(), 0.01))

評価関数 `robotoc.CostFunction` を作成して，上で作成した **cost component** を加えましょう．

In [ ]:
cost = robotoc.CostFunction()
cost.push_back(config_cost)

## Constraints（制約）
制約についても評価関数と同様に作成します．   
すなわち， `robotoc`では様々な**constraint components**を集めて制約, `robotoc.Constraints` (`std::shared_ptr<robotoc::Constraints>` in C++), を作ります．   
各 **constraints component** はある特定の制約を表しており， `robotoc.ConstraintComponentBase` (`robotoc::ConstraintComponentBase` in C++) を継承しています．     
**constraint component** を制約 `robotoc.Constraints` に集め， 制約 `robotoc.Constraints` をソルバーに渡します．   
これにより，様々な **constraint components** を簡単に組み合わせて制約を設計できます.

基本的な **constraint components** のは既に `robotoc` に多数実装されています.  
この例では，関節角度，角速度，トルクの制約を考えます．     
(ここで，これらの制約には `iiwa14.set_joint_effort_limit()` `iiwa14.set_joint_velocity_limit()` でセットした値が使われます)

In [ ]:
joint_position_lower  = robotoc.JointPositionLowerLimit(iiwa14)
joint_position_upper  = robotoc.JointPositionUpperLimit(iiwa14)
joint_velocity_lower  = robotoc.JointVelocityLowerLimit(iiwa14)
joint_velocity_upper  = robotoc.JointVelocityUpperLimit(iiwa14)
joint_torques_lower   = robotoc.JointTorquesLowerLimit(iiwa14)
joint_torques_upper   = robotoc.JointTorquesUpperLimit(iiwa14)

制約`robotoc.Constraints`を作り，上で作成した **constraint components** を加えます．

In [ ]:
constraints = robotoc.Constraints()
constraints.push_back(joint_position_lower)
constraints.push_back(joint_position_upper)
constraints.push_back(joint_velocity_lower)
constraints.push_back(joint_velocity_upper)
constraints.push_back(joint_torques_lower)
constraints.push_back(joint_torques_upper)

## Optimal Control Problem (OCP) and Solver

ここでは最適制御問題 (OCP) とそのソルバーを作ります．  
まずはホライゾン長さ `T` と離散化のグリッドの数 `N` を設定しましょう． 

In [ ]:
T = 3.0
N = 100

今回扱うロボットマニピュレータは接触や floating baseを伴わないため，効率的にOCPを解くことができます．     
このような場合のOCPを記述する `robotoc.UnconstrOCP` (`robotoc::UnconstrOCP` in C++)　を作ります．

In [ ]:
ocp = robotoc.UnconstrOCP(robot=iiwa14, cost=cost, constraints=constraints, T=T, N=N)

ではこのOCPのソルバーを作ります． 
まず， `robotoc.UnconstrOCPSolver` (`robotoc::UnconstrOCPSolver` in C++)　を試します.  
このソルバーは以下の特徴があります: 
- Direct multiple-shooting 法　と primal-dual interior point 法 （主双対内点法）
- Inverse dynamics を用いた **unconstrained** OCP　 (i.e., no contacts or floating bases) の効率的なアルゴリズム.
- Riccati recursion によるニュートンステップの計算

種々のソルバーのオプションを `robotoc.SolverOptions` により設定できます．(ここではデフォルトのパラメータを用います).  
また，ソルバー作成時に並列計算で用いるスレッドの数 (`nthreads`) を指定します.

In [ ]:
solver_options = robotoc.SolverOptions()
ocp_solver = robotoc.UnconstrOCPSolver(ocp=ocp, solver_options=solver_options, nthreads=4)

## Solve the OCP

初期状態 (`q`, `v`) と初期時刻 (`t`) が以下で与えられるOCP（最適制御問題）を考えます．

In [ ]:
t = 0.0
q = np.array([0.5*np.pi, 0, 0.5*np.pi, 0, 0.5*np.pi, 0, 0.5*np.pi]) 
v = np.zeros(iiwa14.dimv())

まずはソルバーを初期化します．   
以下は初期推定解を上の`q`, `v`に設定します．   
これは非常にシンプルですが高速な収束に割と重要です．

In [ ]:
ocp_solver.set_solution("q", q)
ocp_solver.set_solution("v", v)

また，不等式制約に関してソルバーを初期化します．    
(具体的には，主双対内点法のスラック変数とラグランジュ乗数を初期化しますが，特に内部について意識する必要はありません).

In [ ]:
ocp_solver.init_constraints()

ソルバーの現在の推定解がどの程度最適解に近いか，ということを判断するために，本ソルバーでは **KKT error** という指標を用います．   
これは， Karush–Kuhn–Tucker (KKT) 条件（1次の最適性の必要条件）のl2ノルムです. (最適解でこれは0になります.)   
まず，初期化後の KKT error を見てみます． 

In [ ]:
ocp_solver.KKT_error(t, q, v)

次にOCPを解きます．

In [ ]:
ocp_solver.solve(t, q, v)

KKT error がとても小さくなっていることが分かります．    
(具体的には，`robotoc.SolverOption`で設定した閾値より小さくなっています).

In [ ]:
ocp_solver.KKT_error(t, q, v)

また，各反復での KKT error やステップサイズといった情報も以下で見れます．

In [ ]:
stats = ocp_solver.get_solver_statistics()
print(stats)

最適解は以下のようにして集めることができます．

In [ ]:
q_opt = []
v_opt = []
u_opt = []
for i in range(N):
    s = ocp_solver.get_solution(i)
    q_opt.append(s.q)
    v_opt.append(s.v)
    u_opt.append(s.u)
sN = ocp_solver.get_solution(N)
q_opt.append(sN.q)
v_opt.append(sN.v)

解をプロットしてみます．   
グレーの破線が制約を表しており，最適解は制約を厳密に守っていることがわかります．

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn

plt.rcParams['figure.figsize'] = 10, 9
fig = plt.figure()
ax_q = fig.add_subplot(3, 1, 1)
ax_v = fig.add_subplot(3, 1, 2)
ax_u = fig.add_subplot(3, 1, 3)

# Plot results
ax_q.plot(np.linspace(t, t+T, N+1), q_opt)
ax_v.plot(np.linspace(t, t+T, N+1), v_opt)
ax_u.plot(np.linspace(t, t+T-(T/N), N), u_opt)

# Plot constraints boundaries
ax_v.plot(np.linspace(t, t+T, N+1), np.full(N+1, 0.5*np.pi), linestyle='--', color='gray')
ax_v.plot(np.linspace(t, t+T, N+1), np.full(N+1, -0.5*np.pi), linestyle='--', color='gray')
ax_u.plot(np.linspace(t, t+T-(T/N), N), np.full(N, 50.0), linestyle='--', color='gray')
ax_u.plot(np.linspace(t, t+T-(T/N), N), np.full(N, -50.0), linestyle='--', color='gray')

ax_q.set_xlabel('')
ax_v.set_xlabel('')
ax_u.set_xlabel('t [s]')

ax_q.set_ylabel('q [rad]')
ax_v.set_ylabel('v [rad/s]')
ax_u.set_ylabel('u [Nm]')

また，以下で最適化された軌道を可視化できます．

In [ ]:
viewer = robotoc.utils.TrajectoryViewer(path_to_urdf=path_to_urdf,
                                        viewer_type='meshcat')
viewer.display(T/N, q_opt)

## Try Another Solver

次に，別のソルバー `robotoc.UnconstrParNMPCSolver` (`robotoc::UnconstrParNMPCSolver` in C++) を試してみましょう.   
このソルバーは以下の特徴があります: 
- Direct multiple-shooting 法　と primal-dual interior point 法 （主双対内点法）
- Inverse dynamics を用いた **unconstrained** OCP　 (i.e., no contacts or floating bases) の効率的なアルゴリズム.
- ParNMPC アルゴリズム（並列化ニュートン法） によるニュートンステップの計算
- 各ニュートン反復はCPUのコア数が多ければ高速になります．  
- 一方で，収束性能（収束までに必要な反復回数）は先ほどの `robotoc.UnconstrOCPSolver` と比べると悪化します． これはまた，ソルバーのロバスト性も悪くなることを意味します．

種々のソルバーのオプションを `robotoc.SolverOptions` により設定できます．(ここではデフォルトのパラメータを用います).  
また，ソルバー作成時に並列計算で用いるスレッドの数 (`nthreads`) を指定します.

In [ ]:
solver_options = robotoc.SolverOptions()
parnmpc = robotoc.UnconstrParNMPC(robot=iiwa14, cost=cost, constraints=constraints, T=T, N=N)
parnmpc_solver = robotoc.UnconstrParNMPCSolver(parnmpc=parnmpc, solver_options=solver_options, nthreads=8)

In [ ]:
parnmpc_solver.set_solution("q", q)
parnmpc_solver.set_solution("v", v)
parnmpc_solver.init_constraints()

In [ ]:
parnmpc_solver.solve(t, q, v)
stats = parnmpc_solver.get_solver_statistics()
print(stats)